# Using NannyML Performance Estimation Algorithm from AWS Marketplace 

## Overview

NannyML can estimate the performance of a machine learning model running in production. You can read more about how it works [here](https://nannyml.readthedocs.io/en/stable/how_it_works/performance_estimation.html).


This sample notebook shows you how to use [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o) from AWS Marketplace to estimate the performance of your deployed machine learnign models.

Performance Estimation can work for binary classification, multiclass classification and regression.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o). 

## Contents
1. [Subscribe to the algorithm](#1.-Subscribe-to-the-algorithm)
1. [Prepare dataset](#2.-Prepare-dataset)
	1. [Dataset format expected by the algorithm](#A.-Dataset-format-expected-by-the-algorithm)
	1. [Configure and visualize reference dataset](#B.-Configure-and-visualize-reference-dataset)
	1. [Upload datasets to Amazon S3](#C.-Upload-datasets-to-Amazon-S3)
1. [Train the machine learning algorithm](#3:-Train-the-machine-learning-algorithm)
	1. [Set up environment](#3.1-Set-up-environment)
	1. [Train a model](#3.2-Train-the-algorithm)
1. [Deploy model](#4:-Deploy-model)
1. [Perform Batch Inference](#5.-Perform-Batch-Inference)
1. [Clean-up](#6.-Clean-up)
	1. [Delete the model](#A.-Delete-the-model)
	1. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the algorithm

To subscribe to the algorithm:
1. Open the algorithm listing page [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o)
1. On the AWS Marketplace listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify while training a custom ML model. Copy the ARN corresponding to your region and specify the same in the following cell.

<font color='red'>Directly copy your assigned ARN code below:<font>

In [1]:
algo_arn = "<Customer to specify algorithm ARN corresponding to their AWS region>"
# Internally used arn.txt to specify arn to run the notebook
# with open('arn.txt', 'r') as file:
#     algo_arn = file.read().rstrip()

# Show specified ARN
# algo_arn

## 2. Prepare dataset

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
import pandas as pd
import json

### A. Dataset format expected by the algorithm

To fully demonstrate the capabilities of NannyML's performance estimation we will provide code for all 3 supported machine learning problem types.

- For Binary Classification we are going to use [NannyML's synthetic car loan dataset](https://nannyml.readthedocs.io/en/stable/datasets/binary_car_loan.html).
- For Multiclass Classification we are going to use [NannyML's synthetic multiclass creadit card assignment dataset](https://nannyml.readthedocs.io/en/stable/datasets/multiclass.html).
- For Regression we are going to use [NannyML's synthetic car price dataset](https://nannyml.readthedocs.io/en/stable/datasets/regression.html).


You can find some information about dataset format in **Usage Information** section of [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o).
<br>
More detailed information can be found in [NannyML's Data Requirements Documentation](https://nannyml.readthedocs.io/en/stable/tutorials/data_requirements.html).

<font color='red'>Edit code below as appropriate for the Machine Learning problem type you are interested in:<font>

In [3]:
# machine_learning_problem_type = "Binary Classification"
machine_learning_problem_type = "Multiclass Classification"
# machine_learning_problem_type = "Regression"

### B. Configure and visualize reference dataset

In [4]:
if machine_learning_problem_type == "Binary Classification":
    reference_dataset = "data/bc_reference.csv"
elif machine_learning_problem_type == "Multiclass Classification":
    reference_dataset = "data/mc_reference.csv"
elif machine_learning_problem_type == "Regression":
    reference_dataset = "data/reg_reference.csv"
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

# Show selected dataset
pd.read_csv(reference_dataset).head()

,acq_channel,app_behavioral_score,requested_credit_limit,app_channel,credit_bureau_score,stated_income,is_customer,timestamp,y_pred_proba_prepaid_card,y_pred_proba_highstreet_card,y_pred_proba_upmarket_card,y_pred,y_true
0,Partner3,1.808232,350,web,309,15000,True,2020-05-02 02:01:30,0.97,0.03,0.00,prepaid_card,prepaid_card
1,Partner2,4.382568,500,mobile,418,23000,True,2020-05-02 02:03:33,0.87,0.13,0.00,prepaid_card,prepaid_card
2,Partner2,-0.787575,400,web,507,24000,False,2020-05-02 02:04:49,0.47,0.35,0.18,prepaid_card,upmarket_card
3,Partner3,-2.131770,300,mobile,324,38000,False,2020-05-02 02:07:59,0.26,0.50,0.24,highstreet_card,upmarket_card
4,Partner3,-1.362938,450,mobile,736,38000,True,2020-05-02 02:20:19,0.03,0.04,0.93,upmarket_card,upmarket_card


### C. Upload datasets to Amazon S3

In [5]:
sagemaker_session = sage.Session()
bucket = sagemaker_session.default_bucket()
# Uncomment below to see selected default bucket
# bucket

In [6]:
demo_prefix = "doc-notebook-demo"

In [7]:
reference_data = sagemaker_session.upload_data(
    reference_dataset, bucket=bucket, key_prefix=demo_prefix
)

## 3: Train the machine learning algorithm

Now that dataset is available in an accessible Amazon S3 bucket, we are ready to train a machine learning model. 

### 3.1 Set up environment

In [8]:
role = get_execution_role()

In [9]:
output_location = f"s3://{bucket}/{demo_prefix}/output"
# Uncomment below to see specified output location for Algorithm artifacts and outputs
# output_location

### 3.2 Train the algorithm

You can also find more information about dataset format in **Hyperparameters** section of [Model Performance Estimation - NannyML](https://aws.amazon.com/marketplace/pp/prodview-uotyt66szg34o).

For even more detailed information you read NannyML tutorials on performance estimation for:
- [Binary Classification](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/binary_performance_estimation.html)
- [Multiclass Classification](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/multiclass_performance_estimation.html)
- [Regression](https://nannyml.readthedocs.io/en/stable/tutorials/performance_estimation/regression_performance_estimation.html)

In [10]:
# Define hyperparameters
if machine_learning_problem_type == "Binary Classification":
    nannyml_parameters = {
        "y_pred_proba": "y_pred_proba",
        "y_pred": "y_pred",
        "y_true": "repaid",
        "timestamp_column_name": "timestamp",
        "metrics": ["roc_auc"],
        "chunk_size": 5000,
        "problem_type": "classification_binary",
    }
    # json.dumps needed due to sagemaker specifications
    sagemaker_hyperparameters = {
        "data_filename": reference_dataset.split("/")[-1],
        "data_type": "csv",
        "problem_type": "classification_binary",
        "parameters": json.dumps(nannyml_parameters),
    }
elif machine_learning_problem_type == "Multiclass Classification":
    nannyml_parameters = {
        "y_pred": "y_pred",
        "y_pred_proba": {
            "prepaid_card": "y_pred_proba_prepaid_card",
            "highstreet_card": "y_pred_proba_highstreet_card",
            "upmarket_card": "y_pred_proba_upmarket_card"
        },
        "y_true": "y_true",
        "timestamp_column_name": "timestamp",
        "metrics": ["roc_auc"],
        "chunk_size": 5000,
        "problem_type": "classification_multiclass",
    }
    # json.dumps needed due to sagemaker specifications
    sagemaker_hyperparameters = {
        "data_filename": "mc_reference.csv",
        "data_type": "csv",
        "problem_type": "classification_multiclass",
        "parameters": json.dumps(nannyml_parameters),
    }
elif machine_learning_problem_type == "Regression":
    nannyml_parameters = {
        "feature_column_names": [
            "car_age",
            "km_driven",
            "price_new",
            "accident_count",
            "door_count",
            "fuel",
            "transmission",
        ],
        "y_pred": "y_pred",
        "y_true": "y_true",
        "timestamp_column_name": "timestamp",
        "metrics": ["rmse"],
        "chunk_size": 6000,
        "tune_hyperparameters": False,
    }
    # json.dumps needed due to sagemaker specifications
    sagemaker_hyperparameters = {
        "data_filename": reference_dataset.split("/")[-1],
        "data_type": "csv",
        "problem_type": "regression",
        "parameters": json.dumps(nannyml_parameters),
    }
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

For information on creating an `Estimator` object, see [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)

In [11]:
# Create an estimator object for running a training job
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name='nml-perf-est',
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    hyperparameters=sagemaker_hyperparameters,
)

In [12]:
# Run the training job.
estimator.fit(
    {'training': reference_data}
)

INFO:sagemaker:Creating training-job with name: nml-perf-est-2023-08-21-06-17-54-258


2023-08-21 06:17:54 Starting - Starting the training job...
2023-08-21 06:18:10 Starting - Preparing the instances for training......
2023-08-21 06:19:04 Downloading - Downloading input data...
2023-08-21 06:19:29 Training - Downloading the training image...
2023-08-21 06:20:20 Uploading - Uploading generated training modelINFO:nannyml:Logger object created.
INFO:nannyml:Hyperparameters read.
INFO:nannyml:Estimator Instantiated.
INFO:nannyml:Loaded data.
INFO:nannyml.base:fitting nannyml.performance_estimation.confidence_based.cbpe.CBPE
DEBUG:nannyml.usage_logging:found NML_DISABLE_USAGE_LOGGING key in environment variables. Usage event CBPE estimator run not logged.
DEBUG:nannyml.usage_logging:found NML_DISABLE_USAGE_LOGGING key in environment variables. Usage event CBPE estimator fit not logged.
INFO:nannyml:Estimator fit.
INFO:nannyml.io.store.base:storing object "nannyml.performance_estimation.confidence_based.cbpe.CBPE" to store "nannyml.io.store.file_store.FilesystemStore"
DEBUG:

See this [blog-post](https://aws.amazon.com/blogs/machine-learning/easily-monitor-and-visualize-metrics-while-training-models-on-amazon-sagemaker/) for more information how to visualize metrics during the process. You can also open the training job from [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/home?#/jobs/) and monitor the metrics/logs in **Monitor** section.

## 4: Deploy model

**NannyML's Performance Estimation is not designed for real time inference, therefore it is not recommended to use it in this way.**

For this reason we are not showcasing the real-time inference feature of Sagemaker Algorithm.

## 5. Perform Batch Inference

In this section, you will perform batch inference using multiple input payloads together.

In [13]:
# upload the batch-transform job input files to S3

if machine_learning_problem_type == "Binary Classification":
    inference_dataset = "data/bc_analysis.csv"
elif machine_learning_problem_type == "Multiclass Classification":
    inference_dataset = "data/mc_analysis.csv"
elif machine_learning_problem_type == "Regression":
    inference_dataset = "data/reg_analysis.csv"
else:
    raise ValueError("Unsupported Machine Learning Problem Type.")

inference_data = sagemaker_session.upload_data(inference_dataset, bucket=bucket, key_prefix=demo_prefix)
# print("Transform input uploaded to " + inference_data)

In [14]:
# Run the batch-transform job
transformer = estimator.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=output_location
)
transformer.transform(inference_data, content_type="text/csv")
transformer.wait()

INFO:sagemaker:Creating model package with name: nannyml-performance-estimation-1350cd6b-2023-08-21-06-21-07-290


.........

INFO:sagemaker:Creating model with name: nannyml-performance-estimation-1350cd6b-2023-08-21-06-21-52-799


INFO:sagemaker:Creating transform job with name: nml-perf-est-2023-08-21-06-21-55-859


...........................
 * Serving Flask app '/web_app_serve.py'
 * Debug mode: off
#033#033WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.#033
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://127.0.0.1:8080
#033Press CTRL+C to quit#033
169.254.255.130 - - [21/Aug/2023 06:26:31] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [21/Aug/2023 06:26:31] "#033GET /execution-parameters HTTP/1.1#033" 404 -
Received POST invocation request.
Estimation invoked with 60000 rows
Estimation invoked with columns: ['acq_channel', 'app_behavioral_score', 'requested_credit_limit', 'app_channel', 'credit_bureau_score', 'stated_income', 'is_customer', 'timestamp', 'y_pred_proba_prepaid_card', 'y_pred_proba_highstreet_card', 'y_pred_proba_upmarket_card', 'y_pred']
169.254.255.130 - - [21/Aug/2023 06:26:32] "POST /invocations HTTP/1.1" 200 -
2023-08-21T06:26:31.801:[sagemaker logs]: MaxCon

**View Results of Performance Estimation**

In [15]:
results = pd.read_csv(transformer.output_path + "/" + inference_dataset.split("/")[-1] + ".out", header = [0,1])
results

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


chunk                                                          
              key chunk_index start_index end_index           start_date   
0        [0:4999]           0           0      4999  2020-05-02 02:01:30  \
1     [5000:9999]           1        5000      9999  2020-05-12 13:14:18   
2   [10000:14999]           2       10000     14999  2020-05-22 13:41:07   
3   [15000:19999]           3       15000     19999  2020-06-01 18:59:40   
4   [20000:24999]           4       20000     24999  2020-06-11 19:55:34   
5   [25000:29999]           5       25000     29999  2020-06-21 20:51:22   
6   [30000:34999]           6       30000     34999  2020-07-02 02:06:56   
7   [35000:39999]           7       35000     39999  2020-07-12 05:35:07   
8   [40000:44999]           8       40000     44999  2020-07-22 12:58:29   
9   [45000:49999]           9       45000     49999  2020-08-01 10:16:21   
10  [50000:54999]          10       50000     54999  2020-08-11 19:44:49   
11  [55000:59999]          11       55000     59999  2020-08-22 02:53:03   
12       [0:4999]           0           0      4999  2020-09-01 03:10:01   
13    [5000:9999]           1        5000      9999  2020-09-11 14:53:29   
14  [10000:14999]           2       10000     14999  2020-09-21 15:46:29   
15  [15000:19999]           3       15000     19999  2020-10-02 02:45:52   
16  [20000:24999]           4       20000     24999  2020-10-12 07:31:43   
17  [25000:29999]           5       25000     29999  2020-10-22 14:21:04   
18  [30000:34999]           6       30000     34999  2020-11-01 22:04:59   
19  [35000:39999]           7       35000     39999  2020-11-12 02:42:07   
20  [40000:44999]           8       40000     44999  2020-11-22 08:59:10   
21  [45000:49999]           9       45000     49999  2020-12-02 15:49:34   
22  [50000:54999]          10       50000     54999  2020-12-12 17:22:06   
23  [55000:59999]          11       55000     59999  2020-12-22 17:08:38   

                                     roc_auc                            
               end_date     period     value sampling_error  realized   
0   2020-05-12 13:12:37  reference  0.905532       0.002348  0.904142  \
1   2020-05-22 13:37:58  reference  0.909840       0.002348  0.907698   
2   2020-06-01 18:55:46  reference  0.906800       0.002348  0.907093   
3   2020-06-11 19:49:03  reference  0.906455       0.002348  0.909209   
4   2020-06-21 20:51:08  reference  0.907449       0.002348  0.905651   
5   2020-07-02 01:58:05  reference  0.908353       0.002348  0.910833   
6   2020-07-12 05:34:14  reference  0.907318       0.002348  0.903115   
7   2020-07-22 12:54:57  reference  0.908308       0.002348  0.908555   
8   2020-08-01 10:14:52  reference  0.906435       0.002348  0.908082   
9   2020-08-11 19:42:35  reference  0.908585       0.002348  0.907402   
10  2020-08-22 02:52:23  reference  0.907106       0.002348  0.903066   
11  2020-09-01 03:03:23  reference  0.906334       0.002348  0.911346   
12  2020-09-11 14:48:12   analysis  0.907264       0.002348       NaN   
13  2020-09-21 15:46:11   analysis  0.908891       0.002348       NaN   
14  2020-10-02 02:44:16   analysis  0.909456       0.002348       NaN   
15  2020-10-12 07:31:24   analysis  0.911442       0.002348       NaN   
16  2020-10-22 14:13:27   analysis  0.908264       0.002348       NaN   
17  2020-11-01 22:04:40   analysis  0.906829       0.002348       NaN   
18  2020-11-12 02:41:12   analysis  0.819675       0.002348       NaN   
19  2020-11-22 08:58:46   analysis  0.818623       0.002348       NaN   
20  2020-12-02 15:42:32   analysis  0.820996       0.002348       NaN   
21  2020-12-12 17:16:54   analysis  0.818481       0.002348       NaN   
22  2020-12-22 17:07:15   analysis  0.821284       0.002348       NaN   
23  2021-01-01 22:57:55   analysis  0.821463       0.002348       NaN   

                                                                         
   upper_confidence_boundary lower_confidence_boundary upper_t

## 6. Clean-up

### A. Delete the model

In [16]:
transformer.delete_model()

INFO:sagemaker:Deleting model with name: nannyml-performance-estimation-1350cd6b-2023-08-21-06-21-52-799


### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the algorithm, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

